## Web Scraping IMDb
### Programming Paradigms - Team 2

In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

#### Creating a temporary store for data before writing into a file

In [2]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [16]:
# Creating a list of pages as only 50 movies are stored per page on IMDb
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

#### The scraping part

In [5]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

#### Making a dataframe and cleaning it

In [6]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Zack Snyder's Justice League,(2021),242 min,8.2,54,"246,401",-
1,Avengers: Endgame,(2019),181 min,8.4,78,"850,365",$858.37M
2,Sound of Metal,(2019),120 min,7.8,82,"54,497",-
3,The Trial of the Chicago 7,(2020),129 min,7.8,76,"124,229",-
4,Minari,(2020),115 min,7.6,89,"26,144",-


In [7]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

In [8]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Zack Snyder's Justice League,2021,242 min,8.2,54,"246,401",-
1,Avengers: Endgame,2019,181 min,8.4,78,"850,365",$858.37M
2,Sound of Metal,2019,120 min,7.8,82,"54,497",-


In [9]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Zack Snyder's Justice League,2021,242,8.2,54,"246,401",-
1,Avengers: Endgame,2019,181,8.4,78,"850,365",$858.37M
2,Sound of Metal,2019,120,7.8,82,"54,497",-


In [10]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [11]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Zack Snyder's Justice League,2021,242,8.2,54.0,246401,-
1,Avengers: Endgame,2019,181,8.4,78.0,850365,$858.37M
2,Sound of Metal,2019,120,7.8,82.0,54497,-


In [12]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Zack Snyder's Justice League,2021,242,8.2,54.0,246401,NaN
1,Avengers: Endgame,2019,181,8.4,78.0,850365,858.37
2,Sound of Metal,2019,120,7.8,82.0,54497,NaN


In [13]:
movies.dtypes

movie             object
year               int32
time_minute        int32
imdb_rating      float64
metascore        float64
vote               int32
gross_earning    float64
dtype: object

In [14]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Zack Snyder's Justice League,2021,242,8.2,54.0,246401,NaN
1,Avengers: Endgame,2019,181,8.4,78.0,850365,858.37
2,Sound of Metal,2019,120,7.8,82.0,54497,NaN
3,The Trial of the Chicago 7,2020,129,7.8,76.0,124229,NaN
4,Minari,2020,115,7.6,89.0,26144,NaN
...,...,...,...,...,...,...,...
995,OMG: Oh My God!,2012,125,8.1,NaN,52250,0.92
996,Lage Raho Munna Bhai,2006,144,8.1,NaN,43458,2.22
997,Vizontele,2001,110,8.0,NaN,34003,NaN
998,The Breath,2009,128,8.0,NaN,32077,NaN


In [15]:
# Storing it to a .csv file
movies.to_csv('movies.csv')